In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sort_alpha import sorted_alpha
from scipy.spatial import distance

In [ ]:
def torso_size(data):
    neck = [data['x'].loc[1], data['y'].loc[1]]
    if_hip_missing = [data['x'].min(), data['y'].max()]
    if neck ==0:
        torso=0
        return(torso)
    hip1 = [data['x'].loc[8], data['y'].loc[8]]
    hip2 = [data['x'].loc[11], data['y'].loc[11]]
    if hip1[0]>0 and hip2[0]>0:
        probabilities = [data['p'].loc[8], data['p'].loc[11]]
        max_p = np.argmax(probabilities)
        if max_p==0:
            torso = distance.euclidean(hip1, neck)
        else:
            torso = distance.euclidean(hip2, neck)
    elif hip1[0]==0 and hip2[0]>0:
        torso = distance.euclidean(hip2, neck)
    elif hip2[0]==0 and hip1[0]>0:
        torso = distance.euclidean(hip1, neck)
    else:
        torso = distance.euclidean(if_hip_missing, neck)
    return(torso)

In [ ]:
def shoulder_size(data):
    right_shoulder = [data['x'].loc[2], data['y'].loc[2]]
    left_shoulder = [data['x'].loc[5], data['y'].loc[5]]
    if right_shoulder[0] ==0:
        length = distance.euclidean([data['x'].min(), data['y'].max()] ,left_shoulder)
    elif left_shoulder[0] == 0:
        length = distance.euclidean([data['x'].min(), data['y'].max()] ,right_shoulder)
    elif right_shoulder[0]>0 and left_shoulder[0]>0:
        length = distance.euclidean(right_shoulder, left_shoulder)
    else:
        length=0   
    return(length)

In [ ]:
def find_min_round(array):
    array = np.array(array.loc[[0,1,2,5,8,11,15,16,17]])#remove the arms from the equation cause the bounding boxes are too large 
    x = round(np.min(array[np.nonzero(array)]))
    return(x)

def find_max_round(array):
    array = np.array(array.loc[[0,1,2,5,8,11,15,16,17]])
    x = round(np.max(array))
    return(x)

In [ ]:
path_folders = Path('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_tracked_missing_joints_2')
folders = sorted_alpha(path_folders)

In [ ]:
for folder in tqdm(folders): ###outermost loop thru the videos
    while len(sorted_alpha(os.path.join(path_folders,folder))[1:])>2: ### check each video/folder has more than 2 skeletons
        skeletons=[]
        all_sizes = [] ### initiate list of all torsos in the video
        for skeleton in sorted_alpha(os.path.join(path_folders, folder))[1:]: ## loop thru skeletons in video/folder
            body_sizes = []
            skeletons.append(skeleton) ## get a list of skeleton folders inside video
            for coords in sorted_alpha(os.path.join(path_folders, folder, skeleton)): ### loop thru coordinates inside skeleton
                data = pd.read_csv(os.path.join(path_folders, folder, skeleton, coords))
                shoulder = shoulder_size(data)
                torso = torso_size(data)
                body = [shoulder,torso]
                body_sizes.append(np.average(body, weights=[7./10 ,3./10])) ## for each skeleton folder get a list of torso sizes
            all_sizes.append(np.mean(body_sizes,dtype=np.float32)) ## a list of mean torso sizes for each skeleton folder
                                       
        dist = np.subtract(all_sizes, all_sizes[0]) ## compute distance btwn first skel folder and the others
        dist[dist==0]='nan'
                                       
        for file in sorted_alpha(os.path.join(path_folders, folder, skeletons[0])): ## loop thru coordinates in first skel folder
            src_file = Path(os.path.join(path_folders, folder, skeletons[0], file))
            dest_file = Path(os.path.join(path_folders,folder, skeletons[np.nanargmin(dist)], file))
            if dest_file.is_file():
                dest_file = os.path.join(dest_file.parent, f"{(dest_file).stem}_1"+dest_file.suffix)
                shutil.move(src_file,dest_file) ## move each coord file to closest skel folder in terms of torso size
            else:
                shutil.move(src_file,dest_file)
        shutil.rmtree(src_file.parent)   ## delete original skel folder 

print('success')


In [ ]:
type(folders.remove('1043_meal'))

In [ ]:
folders.pop(0)

In [ ]:
folders[1:]